<a href="https://colab.research.google.com/github/abhinav9629/FakeNewsClassifier/blob/main/Fake_News_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [48]:
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import nltk 
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
data = pd.read_csv('/content/drive/MyDrive/DATASETS/NEWS/train.csv')

In [19]:
data.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [20]:
data = data[['text','label']]

In [22]:
data = data.dropna()
data.reset_index(inplace=True)

In [23]:
X = data['text']
Y = data['label']

In [31]:
vocab_size = 5000
stemmer = PorterStemmer()

In [34]:
corpus = []
for i in range(len(data['text'])):
  text = re.sub('[a-zA-Z]',' ',data['text'][i])
  text = text.lower()
  text = text.split()
  text = [stemmer.stem(word) for word in text  if not word in stopwords.words('english')]
  text = " ".join(text)
  corpus.append(text)



In [36]:
one_hot_vector = [one_hot(words,vocab_size) for words in corpus]

In [41]:
sent_len = 100

In [42]:
embedded_vectors = pad_sequences(one_hot_vector,maxlen=sent_len) 

In [47]:
X = np.array(embedded_vectors)
Y = np.array(Y)

In [49]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.25)

In [50]:
embedding_features = 50

In [69]:
model = keras.Sequential([
    keras.layers.Embedding(vocab_size,embedding_features,input_length=sent_len),
    keras.layers.Dropout(0.3),
    keras.layers.LSTM(100),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1,activation='sigmoid')
])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 50)           250000    
_________________________________________________________________
dropout_6 (Dropout)          (None, 100, 50)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout_7 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________


In [70]:
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,loss="binary_crossentropy",metrics=["accuracy"])

In [71]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=20,batch_size=32)

Epoch 1/20
487/487 [==============================] - 35s 69ms/step - loss: 0.5374 - accuracy: 0.7402 - val_loss: 0.4232 - val_accuracy: 0.8045
Epoch 2/20
487/487 [==============================] - 33s 68ms/step - loss: 0.3542 - accuracy: 0.8521 - val_loss: 0.3197 - val_accuracy: 0.8729
Epoch 3/20
487/487 [==============================] - 33s 68ms/step - loss: 0.3089 - accuracy: 0.8769 - val_loss: 0.2984 - val_accuracy: 0.8865
Epoch 4/20
487/487 [==============================] - 33s 68ms/step - loss: 0.2885 - accuracy: 0.8829 - val_loss: 0.2828 - val_accuracy: 0.8863
Epoch 5/20
487/487 [==============================] - 33s 68ms/step - loss: 0.2737 - accuracy: 0.8906 - val_loss: 0.2703 - val_accuracy: 0.8946
Epoch 6/20
487/487 [==============================] - 33s 68ms/step - loss: 0.2640 - accuracy: 0.8950 - val_loss: 0.2629 - val_accuracy: 0.8950
Epoch 7/20
487/487 [==============================] - 33s 68ms/step - loss: 0.2572 - accuracy: 0.8963 - val_loss: 0.2552 - val_accuracy: